# Main

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import statsmodels.formula.api as smf
%matplotlib inline


main()

In [24]:
def find_data():
  sents=pd.read_csv('sents_data_final.csv')
  sents=sents.drop('Unnamed: 0',1)
  return sents

# ver como se comportan los retornos dias antes y despues de la publicacion de las noticias
def main():
  sent = find_data()
  rets=[]
  sd = []
  for i in sent.mean()[0:11]*100:
    rets.append(i)
  for i in sent.std()[0:11]:
    sd.append(i)

  rets=np.array(rets)
  cols=['t_5','t_4','t_3','t_2','t_1','t0','t1','t2','t3','t4','t5']
  ret=pd.DataFrame({'rets':rets,'sd':sd},index=cols)
  print(ret)
  retorn(ret)
  metr(ret,sent)
  regr(sent)

# Graficas

In [21]:
def retorn(ret):
  ret['rets'].plot(figsize=(12,6))
  plt.annotate('Elaboración Propia con datos de Kaggle', (0,0), (-35,-30), fontsize=8,
              xycoords='axes fraction', textcoords='offset points', va='top')
  plt.title('Retornos promedio de los dias anteriores/posteriores al anuncio de noticia')
  plt.ylabel('Retorno diario promedio (%)')
  plt.xlabel('Periodo relacionado con la publicación de la noticia')

# Metrica Estadisitca

In [20]:
def metr(ret,sent):
  prior=['rt_5','rt_4','rt_3','rt_2','rt_1','rt_1']
  leng=[]
  for i in prior:
    leng.append(len(sent[i]))
  t_stats=ret.copy()
  t_stats=t_stats.iloc[0:6,:]
  t_stats['rets']=t_stats['rets']/100
  t_stats=t_stats.reset_index()
  t_stats['len']=leng
  for i in list(t_stats.index):
    t_stats.loc[i,'t_stat']=(t_stats.loc[5,'rets']-t_stats.loc[i,'rets'])/np.sqrt(t_stats.loc[5,'sd']**2/t_stats.loc[5,'len']+ t_stats.loc[i,'sd']**2/t_stats.loc[i,'len'])
  print("............................\n Metrica Estadistica\n")
  print(t_stats)

# Regresion


In [19]:
def regr(sent):
  model_week= smf.ols('rt0~ D0  ', data=sent)
  results_week=model_week.fit(cov_type='HC3')
  print(results_week.summary().as_latex)
  grafclas(sent)

def grafclas(sent):
  sent['sent_val'].value_counts().plot(kind='bar', figsize=(12,6))
  plt.annotate('Elaboración Propia con datos de Kaggle', (0,0), (-35,-30), fontsize=8,
              xycoords='axes fraction', textcoords='offset points', va='top')
  plt.title('Clasificación de los dias segun el sentimiento de las noticias')
  plt.ylabel('Número de días')
  plt.xlabel('Clasificación (Positivo/Negativo)')
